In [ ]:
import pandas as pd
import numpy as np

def calculate_stability_metrics(matrix_csv_path, weights_csv_path, output_csv='stability_metrics_calculated.csv'):
    """
    Calculate stability metrics from raw data CSV files
    
    Parameters:
    - matrix_csv_path: Path to matrix CR values CSV
    - weights_csv_path: Path to criterion weights CSV  
    - output_csv: Output file name for calculated metrics
    """
    
    # Load the raw data
    try:
        matrix_df = pd.read_csv(matrix_csv_path)
        weights_df = pd.read_csv(weights_csv_path)
        print(f"✓ Loaded matrix data: {matrix_df.shape}")
        print(f"✓ Loaded weights data: {weights_df.shape}")
        
        # Display column information for debugging
        print(f"Matrix columns: {list(matrix_df.columns)}")
        print(f"Weights columns: {list(weights_df.columns)}")
        
    except FileNotFoundError as e:
        print(f"Error: Could not find file - {e}")
        return None
    
    # Get unique models
    models = matrix_df['Model'].unique()
    print(f"Found models: {list(models)}")
    
    stability_results = []
    
    for model in models:
        print(f"\nProcessing {model}...")
        
        # Get matrix CR data for this model
        model_matrix_data = matrix_df[matrix_df['Model'] == model]
        cr_values = model_matrix_data['CR'].values
        
        # Get weights data for this model
        model_weights_data = weights_df[weights_df['Model'] == model]
        
        if len(cr_values) == 0 or len(model_weights_data) == 0:
            print(f"  Warning: No data found for {model}")
            continue
        
        # Calculate Mean CR
        mean_cr = np.mean(cr_values)
        
        # Calculate CR Range
        cr_range = f"{np.min(cr_values):.3f}-{np.max(cr_values):.3f}"
        
        # Calculate CR Stability (CV of CR)
        cr_std = np.std(cr_values, ddof=1)  # Sample standard deviation
        cr_stability_cv = cr_std / mean_cr if mean_cr != 0 else 0
        
        # Calculate Weight Stability (Mean CV)
        # This is the average of CV values across all criteria for this model
        cv_values = model_weights_data['CV'].values
        weight_stability_mean_cv = np.mean(cv_values)
        
        # Calculate Max Weight Deviation
        # This is the maximum range across all criteria for this model
        available_cols = model_weights_data.columns.tolist()
        
        if 'Range' in available_cols:
            max_weight_deviation = np.max(model_weights_data['Range'].values)
        elif 'Max_Weight' in available_cols and 'Min_Weight' in available_cols:
            ranges = model_weights_data['Max_Weight'] - model_weights_data['Min_Weight']
            max_weight_deviation = np.max(ranges)
        else:
            # Use CV * Mean_Weight as a proxy for range if range data not available
            print(f"  Warning: Range data not available for {model}, using CV*Mean as approximation")
            approximated_ranges = model_weights_data['CV'] * model_weights_data['Mean_Weight']
            max_weight_deviation = np.max(approximated_ranges)
        
        # Compile results
        result = {
            'Model': model,
            'Mean CR': round(mean_cr, 3),
            'CR Range': cr_range,
            'CR Stability (CV)': round(cr_stability_cv, 3),
            'Weight Stability (Mean CV)': round(weight_stability_mean_cv, 3),
            'Max Weight Deviation': round(max_weight_deviation, 3)
        }
        
        stability_results.append(result)
        
        # Print progress
        print(f"  Mean CR: {mean_cr:.3f}")
        print(f"  CR Range: {cr_range}")
        print(f"  CR Stability (CV): {cr_stability_cv:.3f}")
        print(f"  Weight Stability (Mean CV): {weight_stability_mean_cv:.3f}")
        print(f"  Max Weight Deviation: {max_weight_deviation:.3f}")
    
    # Create DataFrame and sort by Weight Stability (Mean CV)
    results_df = pd.DataFrame(stability_results)
    results_df = results_df.sort_values('Weight Stability (Mean CV)')
    
    # Save results
    results_df.to_csv(output_csv, index=False)
    print(f"\n✓ Stability metrics saved to: {output_csv}")
    
    # Display results
    print(f"\nCalculated Stability Metrics:")
    print("=" * 80)
    print(results_df.to_string(index=False))
    
    return results_df

def detailed_analysis(matrix_csv_path, weights_csv_path):
    """
    Provide detailed analysis and verification of calculations
    """
    
    matrix_df = pd.read_csv(matrix_csv_path)
    weights_df = pd.read_csv(weights_csv_path)
    
    print("DETAILED ANALYSIS")
    print("=" * 60)
    
    models = matrix_df['Model'].unique()
    
    for model in models:
        print(f"\n{model}:")
        print("-" * 40)
        
        # Matrix analysis
        model_matrix = matrix_df[matrix_df['Model'] == model]
        cr_values = model_matrix['CR'].values
        print(f"Individual CR values: {[f'{cr:.4f}' for cr in cr_values]}")
        print(f"Number of iterations: {len(cr_values)}")
        print(f"Mean CR: {np.mean(cr_values):.4f}")
        print(f"Std CR: {np.std(cr_values, ddof=1):.4f}")
        print(f"Min CR: {np.min(cr_values):.4f}")
        print(f"Max CR: {np.max(cr_values):.4f}")
        
        # Weight analysis
        model_weights = weights_df[weights_df['Model'] == model]
        cv_values = model_weights['CV'].values
        print(f"Number of criteria: {len(model_weights)}")
        print(f"CV values range: {np.min(cv_values):.3f} to {np.max(cv_values):.3f}")
        print(f"Mean CV (Weight Stability): {np.mean(cv_values):.3f}")
        
        available_cols = model_weights.columns.tolist()
        if 'Range' in available_cols:
            ranges = model_weights['Range'].values
            print(f"Max weight deviation: {np.max(ranges):.3f}")
        elif 'Max_Weight' in available_cols and 'Min_Weight' in available_cols:
            ranges = model_weights['Max_Weight'] - model_weights['Min_Weight']
            print(f"Max weight deviation: {np.max(ranges):.3f}")
        else:
            approximated_ranges = model_weights['CV'] * model_weights['Mean_Weight']
            print(f"Max weight deviation (approximated): {np.max(approximated_ranges):.3f}")

# Example usage
if __name__ == "__main__":
    # File paths - adjust these to your actual file locations
    matrix_file = "llm_ahp_raw_data_matrix_cr_values.csv"
    weights_file = "llm_ahp_raw_data_criterion_weights.csv"
    
    # Calculate stability metrics
    results = calculate_stability_metrics(matrix_file, weights_file)
    
    if results is not None:
        print(f"\n" + "="*80)
        print("VERIFICATION")
        print("="*80)
        
        # Run detailed analysis for verification
        detailed_analysis(matrix_file, weights_file)
        
        print(f"\n" + "="*80)
        print("SUMMARY")
        print("="*80)
        print(f"✓ Processed {len(results)} models")
        print(f"✓ Calculated all stability metrics")
        print(f"✓ Results saved to CSV file")
        print(f"✓ Models ranked by Weight Stability (Mean CV)")